In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import eig
from numpy.linalg import eigvals
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import eigsh

In [25]:
## economy dimension weighting with AHP

# reading data from excel files
economy=[]
address_string=[]
n_questionnaire=9
for i in range(n_questionnaire):
    address_string.append(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\AHP for weighting indicators\Expert answers\Analysis\AHP Questionnaire'+str(i+1)+'.xlsx')
    economy.append(pd.read_excel(address_string[i],sheet_name='اقتصاد',usecols='L:O').dropna(how='all').reset_index(drop=True))

# number of aggregation levels
n_hierarchy_eco=3    
    
# defining function for estimating weights based on comparison matrices'
def AHP_Economy(economy):
    
    # cleaning data and correcting format problem caused by right to left layout in excel
    eco=[]
    inve_list=[]
    inve_list.append(economy.iloc[1:4,0:3])
    inve_list.append(economy.iloc[5:7,0:2])
    inve_list.append(economy.iloc[8:,0:3])

    for j in range(len(inve_list)):
        eco.append(inve_list[j]*0)
        for i in range(len(inve_list[j].columns)):
            eco[j].iloc[:,i]=inve_list[j].iloc[:,-(i+1)]
        eco[j]=np.array(eco[j],dtype=float)
    
    # calculating eigenvectors and eigenvalues of pairwise matrices'
    weight_vector=[0]*len(eco)
    max_eigenvalue=[0]*len(eco)
    temp=[0,0]
    n=[]
    for i in range(len(eco)):
        n.append(len(eco[i]))
        if len(eco[i])<3:
            temp[0]=eco[i][0,0]
            temp[1]=eco[i][1,0]
            weight_vector[i]=np.array(temp)
            max_eigenvalue[i]=2
        else:
            max_eigenvalue[i], weight_vector[i]=eigs(eco[i],k=1,which='LM')
            weight_vector[i]=np.transpose(abs(weight_vector[i].real))[0]     
            max_eigenvalue[i]=max_eigenvalue[i][0].real
    weight_vector=np.hstack(weight_vector)
        
    return (weight_vector,max_eigenvalue,n)

# defining function for calculating inconsistency ratio
def CR(max_eigenvalue,n):
    temp=0.0000
    if n>2 and n<4:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.52
    else:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.89
    return result
        

In [28]:
# defining index (number of questionnaire) to be used in weight and other dataframes
dfindex=[]
for i in range(n_questionnaire):
    dfindex.append('Q'+str(i+1))

# defining container variables (dataframes and lists) for weights, max eigenvalues, inconsistency indices etc.
max_eigenvalue_eco=pd.DataFrame(np.zeros((len(economy),n_hierarchy_eco), dtype=float),index=dfindex,columns=['E1','E2','E3'])
CR_df_eco=pd.DataFrame(np.zeros((len(economy),n_hierarchy_eco),dtype=float),index=dfindex,columns=['E1','E2','E3'])
n_eco=pd.DataFrame(np.zeros((len(economy),n_hierarchy_eco),dtype=float),index=dfindex,columns=['E1','E2','E3'] )
weight_vector_eco=[]

# populating variables from previous step with appropriate values 
for i in range(n_questionnaire):
    weight_vector_eco.append(AHP_Economy(economy[i])[0])
    max_eigenvalue_eco.iloc[i,:]=AHP_Economy(economy[i])[1]
    n_eco.iloc[i,:]=AHP_Economy(economy[i])[2]
    for j in range(n_hierarchy_eco):
        CR_df_eco.iloc[i,j]=(CR(max_eigenvalue_eco.iloc[i,j],n_eco.iloc[i,j]))

# transforming the list of weights into a dataframe for convenience in next steps    
eco_multicol1=['E1']*3+['E2']*2+['E3']*3
eco_multicol2=['AE','MI','EL','NRWurb','NRWrur','Eagr','IE','NRW']
eco_weights=pd.DataFrame(weight_vector_eco, columns=[eco_multicol1,eco_multicol2],index=dfindex)

# normalizng weights derived from each questionnaire so they sum up to unity
for col in ['E1','E2','E3']:
    eco_weights[col]=eco_weights[col].div(eco_weights[col].sum(axis=1),axis=0)

# calculating average weights based on all expert answers
eco_average_weights_all=eco_weights.mean(axis=0)

# creating the appropriate bollean mask for weights dataframe based on inconsistencies in questionnaires
mask=pd.DataFrame(columns=[eco_multicol1,eco_multicol2],index=dfindex)
for row in dfindex:
    for col1 in ['E1','E2','E3']:
        if CR_df_eco.loc[row,col1]<0.1:
            mask.loc[row,col1]=True
        else:
            mask.loc[row,col1]=False
            
# calculating average weights based on consistent expert answers
eco_average_weights_consistent=eco_weights[mask].mean(axis=0)            

In [59]:
## acces dimension weighting with AHP

# reading data from excel files
acces=[]
address_string=[]
n_questionnaire=9
for i in range(n_questionnaire):
    address_string.append(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\AHP for weighting indicators\Expert answers\Analysis\AHP Questionnaire'+str(i+1)+'.xlsx')
    acces.append(pd.read_excel(address_string[i],sheet_name='دسترسی',usecols='M:P').dropna(how='all').reset_index(drop=True))

# number of aggregation levels
n_hierarchy_acc=3    
    
# defining function for estimating weights based on comparison matrices'
def AHP_Acces(acces):
    
    # cleaning data and correcting format problem caused by right to left layout in excel
    acc=[]
    inva_list=[]
    inva_list.append(acces.iloc[1:4,0:3])
    inva_list.append(acces.iloc[5:8,0:3])
    inva_list.append(acces.iloc[9:,0:])

    for j in range(len(inva_list)):
        acc.append(inva_list[j]*0)
        for i in range(len(inva_list[j].columns)):
            acc[j].iloc[:,i]=inva_list[j].iloc[:,-(i+1)]
        acc[j]=np.array(acc[j],dtype=float)
    
    # calculating eigenvectors and eigenvalues of pairwise matrices'
    weight_vector=[0]*len(acc)
    max_eigenvalue=[0]*len(acc)
    temp=[0,0]
    n=[]
    for i in range(len(acc)):
        n.append(len(acc[i]))
        if len(acc[i])<3:
            temp[0]=acc[i][0,0]
            temp[1]=acc[i][1,0]
            weight_vector[i]=np.array(temp)
            max_eigenvalue[i]=2
        else:
            max_eigenvalue[i], weight_vector[i]=eigs(acc[i],k=1,which='LM')
            weight_vector[i]=np.transpose(abs(weight_vector[i].real))[0]     
            max_eigenvalue[i]=max_eigenvalue[i][0].real
        
    return (weight_vector,max_eigenvalue,n)

# defining function for calculating inconsistency ratio
def CR(max_eigenvalue,n):
    temp=0.0000
    if n>2 and n<4:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.52
    else:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.89
    return result


CR_df_acc=pd.DataFrame(np.zeros((len(acces),n_hierarchy_acc), dtype=float))
weight_vector_acc=[[]]*len(acces)
max_eigenvalue_acc=[[]]*len(acces)
n_acc=[[]]*len(acces)

for i in range(len(acces)):
    weight_vector_acc[i]=AHP_Acces(acces[i])[0]
    max_eigenvalue_acc[i]=AHP_Acces(acces[i])[1]
    n_acc[i]=AHP_Acces(acces[i])[2]
    for j in range(len(max_eigenvalue_acc[i])):
        CR_df_acc.iloc[i,j]=(CR(max_eigenvalue_acc[i][j],n_acc[i][j]))
    for k in range(n_hierarchy_acc):
        weight_vector_acc[i][k]=weight_vector_acc[i][k]/weight_vector_acc[i][k].sum()
CR_df_acc

,0,1,2
0,1.078254e+00,3.702989e-02,0.444431
1,0.000000e+00,0.000000e+00,0.127270
2,1.757373e-01,5.392632e-01,0.206260
3,0.000000e+00,2.834418e-01,0.012282
4,5.444289e-01,1.125959e-01,0.728439
5,1.708035e-15,-8.540177e-16,0.000000
6,0.000000e+00,0.000000e+00,0.000000
7,6.751697e-03,3.702989e-02,0.093484
8,1.708035e-15,2.834418e-01,0.115881


In [61]:
#calculating average weights for access dimension only for consistent experts

consistent_experts=list(CR_df_acc[CR_df_acc<0.1].dropna().index)
average_weights_acc_consistent=weight_vector_acc[0]
for i in range(n_hierarchy_acc):
    for k in range(n_acc[0][i]):
        temp=0
        for j in consistent_experts:
            temp=temp+weight_vector_acc[j][i][k]
        avg=temp/len(consistent_experts)
        average_weights_acc_consistent[i][k]=avg
        
#calculating average weights for accsess dimension for all experts
average_weights_acc_all=weight_vector_acc[0]
for i in range(n_hierarchy_acc):
    for k in range(n_acc[0][i]):
        temp=0
        for j in range(n_questionnaire):
            temp=temp+weight_vector_acc[j][i][k]
        avg=temp/n_questionnaire
        average_weights_acc_all[i][k]=avg
average_weights_acc_all

[array([0.34666063, 0.22823665, 0.42510272]),
 array([0.51047023, 0.16608729, 0.32344248]),
 array([0.51983694, 0.12223527, 0.16538361, 0.19254419])]

In [58]:
## resource dimension weighting with AHP

# reading data from excel files
resource=[]
address_string=[]
n_questionnaire=9
for i in range(n_questionnaire):
    address_string.append(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\AHP for weighting indicators\Expert answers\Analysis\AHP Questionnaire'+str(i+1)+'.xlsx')
    resource.append(pd.read_excel(address_string[i],sheet_name='منابع',usecols='L:O').dropna(how='all').reset_index(drop=True))
    
# number of aggregation levels
n_hierarchy_res=7    
    
# defining function for estimating weights based on comparison matrices'
def AHP_Resource(resource):
    
    # cleaning data and correcting format problem caused by right to left layout in excel
    res=[]
    invr_list=[]
    invr_list.append(resource.iloc[1:3,0:2])
    invr_list.append(resource.iloc[4:6,1:3])
    invr_list.append(resource.iloc[7:11,0:])
    invr_list.append(resource.iloc[12:14,1:3])
    invr_list.append(resource.iloc[15:18,0:3])
    invr_list.append(resource.iloc[19:21,1:3])
    invr_list.append(resource.iloc[22:,0:])

    for j in range(len(invr_list)):
        res.append(invr_list[j]*0)
        for i in range(len(invr_list[j].columns)):
            res[j].iloc[:,i]=invr_list[j].iloc[:,-(i+1)]
        res[j]=np.array(res[j],dtype=float)
    
    # calculating eigenvectors and eigenvalues of pairwise matrices'
    weight_vector=[0]*len(res)
    max_eigenvalue=[0]*len(res)
    temp=[0,0]
    n=[]
    for i in range(len(res)):
        n.append(len(res[i]))
        if len(res[i])<3:
            temp[0]=res[i][0,0]
            temp[1]=res[i][1,0]
            weight_vector[i]=np.array(temp)
            max_eigenvalue[i]=2
        else:
            max_eigenvalue[i], weight_vector[i]=eigs(res[i],k=1,which='LM')
            weight_vector[i]=np.transpose(abs(weight_vector[i].real))[0]     
            max_eigenvalue[i]=max_eigenvalue[i][0].real
        
    return (weight_vector,max_eigenvalue,n)

# defining function for calculating inconsistency ratio
def CR(max_eigenvalue,n):
    temp=0.0000
    if n>2 and n<4:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.52
    else:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.89
    return result


CR_df_res=pd.DataFrame(np.zeros((len(resource),n_hierarchy_res), dtype=float))
weight_vector_res=[[]]*len(resource)
max_eigenvalue_res=[[]]*len(resource)
n_res=[[]]*len(resource)

for i in range(len(resource)):
    weight_vector_res[i]=AHP_Resource(resource[i])[0]
    max_eigenvalue_res[i]=AHP_Resource(resource[i])[1]
    n_res[i]=AHP_Resource(resource[i])[2]
    for j in range(n_hierarchy_res):
        CR_df_res.iloc[i,j]=(CR(max_eigenvalue_res[i][j],n_res[i][j]))
    for k in range(n_hierarchy_res):
        weight_vector_res[i][k]=weight_vector_res[i][k]/weight_vector_res[i][k].sum()
CR_df_res

,0,1,2,3,4,5,6
0,0.0,0.0,1.816890e-01,0.0,1.303950e-01,0.0,0.720592
1,0.0,0.0,6.476588e-01,0.0,-8.540177e-16,0.0,0.043244
2,0.0,0.0,2.016782e-01,0.0,0.000000e+00,0.0,0.181983
3,0.0,0.0,1.228218e-02,0.0,0.000000e+00,0.0,0.098877
4,0.0,0.0,4.287504e-01,0.0,2.834418e-01,0.0,0.183766
5,0.0,0.0,-3.326511e-16,0.0,0.000000e+00,0.0,0.000000
6,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000005
7,0.0,0.0,5.024307e-02,0.0,1.708035e-15,0.0,0.022222
8,0.0,0.0,1.457061e-01,0.0,3.702989e-02,0.0,0.025852


In [67]:
#calculating average weights for resource dimension only for consistent experts

consistent_experts=list(CR_df_res[CR_df_res<0.1].dropna().index)
average_weights_res_consistent=weight_vector_res[0]
for i in range(n_hierarchy_res):
    for k in range(n_res[0][i]):
        temp=0
        for j in consistent_experts:
            temp=temp+weight_vector_res[j][i][k]
        avg=temp/len(consistent_experts)
        average_weights_res_consistent[i][k]=avg
        
#calculating average weights for resource dimension for all experts
average_weights_res_all=weight_vector_res[0]
for i in range(n_hierarchy_res):
    for k in range(n_res[0][i]):
        temp=0
        for j in range(n_questionnaire):
            temp=temp+weight_vector_res[j][i][k]
        avg=temp/n_questionnaire
        average_weights_res_all[i][k]=avg


[3, 5, 6, 7]

In [68]:
average_weights_res_all

[array([0.72222222, 0.27777778]),
 array([0.62731481, 0.37268519]),
 array([0.40693962, 0.10169136, 0.29114321, 0.20022581]),
 array([0.34932818, 0.65067182]),
 array([0.35245012, 0.40022965, 0.24732023]),
 array([0.52083333, 0.47916667]),
 array([0.30522472, 0.13624316, 0.32080037, 0.23773175])]

In [69]:
average_weights_res_consistent

[array([0.72222222, 0.27777778]),
 array([0.62731481, 0.37268519]),
 array([0.40693962, 0.10169136, 0.29114321, 0.20022581]),
 array([0.34932818, 0.65067182]),
 array([0.35245012, 0.40022965, 0.24732023]),
 array([0.52083333, 0.47916667]),
 array([0.30522472, 0.13624316, 0.32080037, 0.23773175])]

In [70]:
weight_vector_res[0]

[array([0.72222222, 0.27777778]),
 array([0.62731481, 0.37268519]),
 array([0.40693962, 0.10169136, 0.29114321, 0.20022581]),
 array([0.34932818, 0.65067182]),
 array([0.35245012, 0.40022965, 0.24732023]),
 array([0.52083333, 0.47916667]),
 array([0.30522472, 0.13624316, 0.32080037, 0.23773175])]

In [18]:
## water security index weighting dimensions with AHP

# reading data from excel files
security=[]
address_string=[]
n_questionnaire=9
for i in range(n_questionnaire):
    address_string.append(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\AHP for weighting indicators\Expert answers\Analysis\AHP Questionnaire'+str(i+1)+'.xlsx')
    security.append(pd.read_excel(address_string[i],sheet_name='امنیت آب',usecols='L:O').dropna(how='all').reset_index(drop=True))
    
# number of aggregation levels
n_hierarchy=1    
    
# defining function for estimating weights based on comparison matrices'
def AHP_WSI(security):
    
    # cleaning data and correcting format problem caused by right to left layout in excel
    wsi=[]
    invwsi_list=[]
    invwsi_list.append(security.iloc[1:4,0:3])

    for j in range(len(invwsi_list)):
        wsi.append(invwsi_list[j]*0)
    for i in range(len(invwsi_list[j].columns)):
        wsi[j].iloc[:,i]=invwsi_list[j].iloc[:,-(i+1)]
    wsi[j]=np.array(wsi[j],dtype=float)
    
    # calculating eigenvectors and eigenvalues of pairwise matrices'
    weight_vector=[0]*len(wsi)
    max_eigenvalue=[0]*len(wsi)
    temp=[0,0]
    n=[]
    for i in range(len(wsi)):
        n.append(len(wsi[i]))
        if len(wsi[i])<3:
            temp[0]=wsi[i][0,0]
            temp[1]=wsi[i][1,0]
            weight_vector[i]=np.array(temp)
            max_eigenvalue[i]=2
        else:
            max_eigenvalue[i], weight_vector[i]=eigs(wsi[i],k=1,which='LM')
            weight_vector[i]=np.transpose(abs(weight_vector[i].real))[0]     
            max_eigenvalue[i]=max_eigenvalue[i][0].real
        
    return (weight_vector,max_eigenvalue,n)

# defining function for calculating inconsistency ratio
def CR(max_eigenvalue,n):
    temp=0.0000
    if n>2 and n<4:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.52
    else:
        temp=(max_eigenvalue-n)/(n-1)
        result=temp/0.89
    return result


CR_df_wsi=pd.DataFrame(np.zeros((len(security),n_hierarchy), dtype=float))
weight_vector_wsi=[[]]*len(security)
max_eigenvalue_wsi=[[]]*len(security)
n_wsi=[[]]*len(security)

for i in range(len(security)):
    weight_vector_wsi[i]=AHP_WSI(security[i])[0]
    max_eigenvalue_wsi[i]=AHP_WSI(security[i])[1]
    n_wsi[i]=AHP_WSI(security[i])[2]
    for j in range(len(max_eigenvalue_wsi[i])):
        CR_df_wsi.iloc[i,j]=(CR(max_eigenvalue_wsi[i][j],n_wsi[i][j]))
    for k in range(n_hierarchy):
        weight_vector_wsi[i][k]=weight_vector_wsi[i][k]/weight_vector_wsi[i][k].sum()
weight_vector_wsi
    

[[array([0.61750423, 0.29686507, 0.0856307 ])],
 [array([0.6812769 , 0.24994742, 0.06877568])],
 [array([0.63698557, 0.25828499, 0.10472943])],
 [array([0.71428571, 0.14285714, 0.14285714])],
 [array([0.42816023, 0.14224275, 0.42959702])],
 [array([0.66666667, 0.11111111, 0.22222222])],
 [array([0.6, 0.2, 0.2])],
 [array([0.24263692, 0.66941687, 0.08794621])],
 [array([0.63698557, 0.25828499, 0.10472943])]]